In [2]:
import json
import os
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
with open('C:/Users/jizhouw0/Desktop/sample_transcripts_xml/20000426-16448-C.xml') as f:
    file = f.read()

call = BeautifulSoup(file, 'xml')

In [24]:
# experiment with the meta section
date = call.find('date').text
print(date)

company = call.find_all('company')
for c in company: print(type(company), type(c.text))


2000-04-26
<class 'bs4.element.ResultSet'> <class 'str'>


In [45]:
# experiment with the management discussion section
discussion = call.find('section', {'name': 'MANAGEMENT DISCUSSION SECTION'}).find_all('speaker')
# Note that the first sectence is an introduction and is not part of the earnings call
# Implementation: we wnat to collect all those sentences somewhere -- not important
# print(discussion[0].get('id') is None)

full_discussion = ""
for part in discussion:
    if part.get('id') not in (None, '0'):
        paragraphs = part.find_all('p')
        for paragraph in paragraphs:
            full_discussion = full_discussion + " " + paragraph.text
            
print(full_discussion)
        
    

 Good morning. I am Doug MacMillan, Cambrex Chief Financial Officer. Before we begin, I just want to remind everyone that the meeting is held today under the terms of the Safe Harbor disclaimer. That disclaimer reads as follows: The following discussion may contain certain statements that constitute forward-looking statements within the Securities and Exchange Commission's Safe Harbor provision under the Private Securities Litigation Reform Act and Rule 3B-6 under the Exchange Act. Although such statements represent management's best judgment as of today and that information currently available, such statements involve risks and uncertainties that could cause results to differ materially from the Company's expectations. I would also like to ask everyone to follow a few guidelines. The conference call will be a maximum of one hour and we will begin with business highlights from Jim Mack, Cambrex Chairman and Chief Executive followed by a question-and-answer session. We request that you 

In [2]:
# define directory where the transcripts are stored 
directory = "C:/Users/jizhouw0/Desktop/sample transcripts"

# define a helper function -- line_counter
def line_counter(a_file):
    """Count the number of lines in a text file
    
    Arguments:
        a_file {str or Path} -- input text file
    
    Returns:
        int -- number of lines in the file
    """
    n_lines = 0
    with open(a_file, "rb") as f:
        n_lines = sum(1 for _ in f)
    return n_lines

In [6]:
# This is just a block of junk code to facilitate the visual inspection of data

for filename in os.scandir(directory):
    if filename.is_file():
        with open(filename.path) as f:
            trspt = json.load(f)
            components = trspt["components"]
            for j in trspt.keys():
                print(j)


transcriptid
keydevid
companyid
companyname
headline
transcriptcreationdate
mostimportantdate
components
full_transcript_text
transcriptid
keydevid
companyid
companyname
headline
transcriptcreationdate
mostimportantdate
components
full_transcript_text
transcriptid
keydevid
companyid
companyname
headline
transcriptcreationdate
mostimportantdate
components
full_transcript_text
transcriptid
keydevid
companyid
companyname
headline
transcriptcreationdate
mostimportantdate
components
full_transcript_text
transcriptid
keydevid
companyid
companyname
headline
transcriptcreationdate
mostimportantdate
components
full_transcript_text
transcriptid
keydevid
companyid
companyname
headline
transcriptcreationdate
mostimportantdate
components
full_transcript_text


In [14]:
# We iteratre over all the transripts (in json format) and generate three text files as inout for the ML algorithm 
# scripted section, Q & A section, and both
text_scripted, text_unscripted, text_both = "", "", ""
id2firms = {"document_id":[], "firm_id": [], "time": [], "firm_name": []}
doc_count = 0
for filename in os.scandir(directory):
    if filename.is_file():
        doc_count += 1 # counts the number of documents
        with open(filename.path) as f:
            trspt = json.load(f)
            id2firms["document_id"].append(trspt["transcriptid"])
            id2firms["firm_id"].append(trspt["companyid"])
            id2firms["time"].append(trspt["transcriptcreationdate"])
            id2firms["firm_name"].append(trspt["companyname"])
            
            components = trspt["components"]
            
            str_list_both = [components[i]["text"] for i in range(len(components)) if components[i]["componenttypename"] != "Presentation Operator Message"]
            text_both = text_both + '\n' + "".join(str_list_both).replace('\r', '').replace('\n', '')
            
            str_list_unscripted = [components[i]["text"] for i in range(len(components)) if components[i]["componenttypename"] != "Presentation Operator Message" if components[i]["componenttypename"] != "Presenter Speech"]
            text_unscripted = text_unscripted + '\n' + "".join(str_list_unscripted).replace('\r', '').replace('\n', '')
            
            str_list_scripted = [components[i]["text"] for i in range(len(components)) if components[i]["componenttypename"] == "Presenter Speech"]
            text_scripted = text_scripted + '\n' + "".join(str_list_scripted).replace('\r', '').replace('\n', '')
            
with open('C:/Users/jizhouw0/Desktop/sample transcripts/text_all/transcripts_both.txt', 'w') as f: 
    f.write(text_both.replace("\n","",1))
    
with open('C:/Users/jizhouw0/Desktop/sample transcripts/text_all/transcripts_unscripted.txt', 'w') as f: 
    f.write(text_unscripted.replace("\n","",1))

with open('C:/Users/jizhouw0/Desktop/sample transcripts/text_all/transcripts_scripted.txt', 'w') as f: 
    f.write(text_scripted.replace("\n","",1))
    
document_id = map(str, id2firms["document_id"])
with open('C:/Users/jizhouw0/Desktop/sample transcripts/text_all/document_ids.txt', 'w') as f: 
    f.write('\n'.join(document_id))
    
df_id2firms = pd.DataFrame(data=id2firms)
df_id2firms.to_csv("C:/Users/jizhouw0/Desktop/sample transcripts/text_all/id2firms.csv", header = True, index = False)
    
assert line_counter("C:/Users/jizhouw0/Desktop/sample transcripts/text_all/transcripts_both.txt") == doc_count, "Line Number Differs From Document Count!"
assert df_id2firms.shape[0] == line_counter("C:/Users/jizhouw0/Desktop/sample transcripts/text_all/transcripts_both.txt"), "Number of IDs Differs From Number of Documents!"
            


In [10]:
# We iteratre over all the transripts (in json format) and generate three text files as inout for the ML algorithm 
# scripted section, Q & A section, and both
text_scripted, text_unscripted, text_both = "", "", ""
id2firms = {"document_id":[], "firm_id": [], "time": [], "firm_name": []}
doc_count = 0
for filename in os.scandir(directory):
    if filename.is_file():
        doc_count += 1 # counts the number of documents
        with open(filename.path) as f:
            trspt = json.load(f)
        id2firms["document_id"].append(trspt["transcriptid"])
        id2firms["firm_id"].append(trspt["companyid"])
        id2firms["time"].append(trspt["transcriptcreationdate"])
        id2firms["firm_name"].append(trspt["companyname"])
            
        components = trspt["components"]
            
        str_list_both = [components[i]["text"] for i in range(len(components)) if components[i]["componenttypename"] != "Presentation Operator Message"]
        with open('C:/Users/jizhouw0/Desktop/sample transcripts/text_all/transcripts_both.txt', 'a') as out_f: 
            out_f.write("".join(str_list_both).replace('\r', '').replace('\n', '')) if doc_count == 0 else out_f.write("".join(str_list_both).replace('\r', '').replace('\n', '') + '\n')
            
        str_list_unscripted = [components[i]["text"] for i in range(len(components)) if components[i]["componenttypename"] != "Presentation Operator Message" if components[i]["componenttypename"] != "Presenter Speech"]
        with open('C:/Users/jizhouw0/Desktop/sample transcripts/text_all/transcripts_unscripted.txt', 'a') as out_f: 
            out_f.write("".join(str_list_unscripted).replace('\r', '').replace('\n', '')) if doc_count == 0 else out_f.write("".join(str_list_unscripted).replace('\r', '').replace('\n', '') + '\n')
            
            
        str_list_scripted = [components[i]["text"] for i in range(len(components)) if components[i]["componenttypename"] == "Presenter Speech"]
        with open('C:/Users/jizhouw0/Desktop/sample transcripts/text_all/transcripts_scripted.txt', 'a') as out_f: 
            out_f.write("".join(str_list_scripted).replace('\r', '').replace('\n', '')) if doc_count == 0 else out_f.write("".join(str_list_scripted).replace('\r', '').replace('\n', '') + '\n')

document_id = map(str, id2firms["document_id"])
id2firms["firm_id"] = map(str, id2firms["firm_id"])
with open('C:/Users/jizhouw0/Desktop/sample transcripts/text_all/document_ids.txt', 'w') as f: 
    f.write('\n'.join(document_id))
    
df_id2firms = pd.DataFrame(data=id2firms)
df_id2firms[["document_id", "firm_id"]] = df_id2firms[["document_id", "firm_id"]].astype(str)
df_id2firms.to_csv("C:/Users/jizhouw0/Desktop/sample transcripts/text_all/id2firms.csv", header = True, index = False)
    
assert line_counter("C:/Users/jizhouw0/Desktop/sample transcripts/text_all/transcripts_both.txt") == doc_count, "Line Number Differs From Document Count!"
assert df_id2firms.shape[0] == line_counter("C:/Users/jizhouw0/Desktop/sample transcripts/text_all/transcripts_both.txt"), "Number of IDs Differs From Number of Documents!"
            
